In [47]:
import pandas as pd
import numpy as np
import os
from math import radians, sin, cos, sqrt, atan2

def calculate_distance_between_points(lat1, lon1, lat2, lon2):
    """Calculate the Haversine distance."""
    R = 6371e3  # meters
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    delta_phi = radians(lat2 - lat1)
    delta_lambda = radians(lon2 - lon1)

    a = sin(delta_phi / 2.0) ** 2 + cos(phi1) * cos(phi2) * sin(delta_lambda / 2.0) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

def resample_data(df):
    df.set_index('datetime', inplace=True)
    df_resampled = df.resample('15S').first()  # Reduced time for more frequent data points
    df_resampled['datetime_temp'] = df_resampled.index
    return df_resampled.reset_index(drop=True).rename(columns={'datetime_temp': 'datetime'})

def filter_points_based_on_distance(df):
    filtered_df = df.iloc[[0]]
    last_index = 0
    for i in range(1, len(df)):
        distance = df.iloc[i]['distance']
        accumulated_distance = df.iloc[last_index:i+1]['distance'].sum()
        if accumulated_distance >= 300:
            filtered_df = pd.concat([filtered_df, df.iloc[[i]]])
            last_index = i
    return filtered_df

def filter_within_bounds(df, lat_range=(18.799457040913197, 18.799507014326885), lng_range=(98.95188455231201,  98.95332376421896)):
    """Filter dataframe for lat and lng within the specified ranges."""
    return df[(df['lat'] >= lat_range[0]) & (df['lat'] <= lat_range[1]) &
              (df['lng'] >= lng_range[0]) & (df['lng'] <= lng_range[1])]

def detect_and_remove_long_stops(df, min_stop_duration=10*60):  # default 10 minutes
    """Detects long stops and removes them from DataFrame"""
    stopped_df = df[df['velocity_kph'] == 0]
    start_stops = stopped_df[stopped_df['time_diff'] > pd.Timedelta(minutes=10)]
    stop_indices = start_stops.index.tolist()
    
    for idx in stop_indices:
        if idx - 1 not in stop_indices:
            stop_indices.append(idx - 1)
    stop_indices.sort()
    
    df = df.drop(stop_indices)
    return df

def calculate_velocity(file_path):
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'], format='%m/%d/%Y %I:%M:%S %p')

    # Filtering by bounds first to reduce subsequent processing
    df = filter_within_bounds(df)
    
    df = resample_data(df)
    
    df['lat_prev'] = df['lat'].shift(1)
    df['lng_prev'] = df['lng'].shift(1)
    
    df['distance'] = df.apply(
        lambda row: calculate_distance_between_points(row['lat'], row['lng'], row['lat_prev'], row['lng_prev']), 
        axis=1
    )
    
    df['time_diff'] = df['datetime'].diff().fillna(pd.Timedelta(seconds=0))
    df['velocity_mps'] = df.apply(lambda row: row['distance'] / row['time_diff'].total_seconds() if row['time_diff'].total_seconds() > 0 else 0, axis=1)
    df['velocity_kph'] = (df['velocity_mps'] * 3.6).round(2)
    df['velocity_kph'].fillna(-1, inplace=True)
    df['velocity_kph'] = df['velocity_kph'].astype(int)
    df['lat'].interpolate(inplace=True)
    df['lng'].interpolate(inplace=True)
    df['accumulated_distance'] = df['distance'].cumsum()
    
    df = detect_and_remove_long_stops(df)
    df = filter_points_based_on_distance(df)

    pd.options.display.float_format = '{:,.2f}'.format

    return df[['datetime', 'lat', 'lng', 'velocity_mps', 'velocity_kph']]
"""
def average_speeds_by_time(df):
    # Extract day of week and hour from datetime
    df['day_of_week'] = df['datetime'].dt.day_name()
    df['hour'] = df['datetime'].dt.hour

    # Filter dataframe based on hrs. range
    hour_ranges = [(7, 8), (8, 9), (9, 10), (10, 11), (11, 12),
                   (12, 13), (13, 14), (14, 15), (15, 16), (16, 17),
                   (17, 18), (18, 19), (19, 20), (20, 21), (21, 22)]
    
    mask = df['hour'].isin([start for start, _ in hour_ranges])
    df_filtered = df[mask]

    # Group by day_of_week and hour,and the mean velocity
    grouped = df_filtered.groupby(['day_of_week', 'hour']).agg(average_speed=('velocity_kph', 'mean'))
    return grouped
"""
"""
def average_speeds_by_day_of_month(df, intervals):
    # Extract day and hour from datetime
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    
    # Create a list to store results
    results = []

    # Iterate over each time interval
    for start, end in intervals:
        mask = (df['hour'] >= start) & (df['hour'] < end)
        df_filtered = df[mask]
        
        # Group by day and compute the mean velocity for the time interval
        grouped = df_filtered.groupby('day').agg(average_speed=('velocity_kph', 'mean')).reset_index()
        grouped['time_interval'] = f"{start}-{end}"
        
        # Ensure all days of the month are present
        all_days = set(range(1, 32))
        missing_days = all_days - set(grouped['day'])
        for day in missing_days:
            missing_row = pd.DataFrame([{'day': day, 'average_speed': np.nan, 'time_interval': f"{start}-{end}"}])
            grouped = pd.concat([grouped, missing_row]).reset_index(drop=True)

        results.append(grouped)
    
    final_df = pd.concat(results).pivot(index='day', columns='time_interval', values='average_speed')
    
    # Sort the final dataframe by day for readability
    final_df = final_df.sort_index()

    return final_df
"""
def average_speeds_by_day_of_month(df, intervals):
    # Extract day and hour from datetime
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    
    # Create a list to store results
    results = []

    # Iterate over each time interval
    for start, end in intervals:
        mask = (df['hour'] >= start) & (df['hour'] < end)
        df_filtered = df[mask]
        
        # Group by day and compute the mean velocity for the time interval
        grouped = df_filtered.groupby('day').agg(average_speed=('velocity_kph', 'mean')).reset_index()
        grouped['time_interval'] = f"{start}-{end}"
        
        results.append(grouped)
    
    final_df = pd.concat(results).pivot(index='day', columns='time_interval', values='average_speed')
    
    return final_df


# Define the time intervals
hour_intervals = [(7, 8), (8, 9), (9, 10), (10, 11), (11, 12),
                  (12, 13), (13, 14), (14, 15), (15, 16), (16, 17),
                  (17, 18), (18, 19), (19, 20), (20, 21), (21, 22)]

def get_month_from_filename(filename):
    """Extracts month from filename in format 'Influx_YYYY_MM.csv'"""
    return filename.split('_')[2]
'''
def process_datasets(directory_path):
    all_results = []
    
    # List all files in directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            
            # Get full file path
            file_path = os.path.join(directory_path, filename)
            
            # Compute velocity for the dataset
            result_df = calculate_velocity(file_path)
            
            # Calculate average speeds by day of the month
            avg_speeds_df = average_speeds_by_day_of_month(result_df, hour_intervals)
            
            # Add month column
            avg_speeds_df['month'] = get_month_from_filename(filename)
            
            # Append result to all_results list
            all_results.append(avg_speeds_df)
    
    # Concatenate all DataFrames in the list
    final_df = pd.concat(all_results, ignore_index=True)
    return final_df
'''

result_df = calculate_velocity('Influx_2022_12.csv')

#average_speed_df = average_speeds_by_time(result_df)
avg_speeds_df = average_speeds_by_day_of_month(result_df, hour_intervals)
'''
directory_path = "/home/sorawich_y/ProjSurvey"
final_result = process_datasets(directory_path)'''
#print(final_result)

print(avg_speeds_df)


print(result_df)

time_interval  10-11  11-12  12-13  13-14  14-15   7-8   8-9  9-10
day                                                               
1              23.25   8.75  12.80   9.67  11.20  5.75 10.00 10.00
2              12.40   6.33   7.00  10.00   9.00 14.25 16.00 12.00
3              18.33   8.00  10.50  11.67   8.67  9.50 17.50 15.50
4              10.50  15.33   9.33  14.67   8.00 10.00 17.50 12.25
5               4.00   8.67  16.33  13.00   8.67 11.67  7.00 15.60
6              11.00  15.20  18.00  10.62  14.25  3.00 15.43 12.80
7               9.80  15.00  11.80  14.60  20.00 10.75  9.20 11.62
8               9.33   7.67  11.00  12.00  19.00 21.00 10.00  9.29
9              10.75  10.00  12.75  14.86   9.25 15.67  5.00  8.40
10              5.67   6.67  12.80  10.75  10.67 16.00 13.00 10.33
11              9.75   6.50  10.60   8.00  14.33 12.50 16.50 10.50
12             13.00  19.83  14.67   7.00  10.67 14.20 12.75 19.75
13              7.00  10.40  11.50   9.50   8.67  9.25 14.80 1